notes on using the gaia astrophysical parameters:

Teff is returned via:
* astrophysical_parameters.teff_gspphot, astrophysical_parameters.teff_gspspec 

* gspphot is based on the low res gaia spectroscopy
* gspspec is based on the high res RV spectroscopy

RV spectroscopy is available for many of these bright sources

gspspec does not seem to return Teff above ~7000 K very reliably

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
#import astropy.table as table
from astropy.table import Table
import time

from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
from astroquery.gaia import Gaia

# fixed instrument parameters
lvm_fiber_diameter = 35.3 # arcsec
lvm_fiber_separation = 37.0 #arcsec
lvm_nrings = 5 #for sky telescope

FIBER_RADIUS_ARCSEC = lvm_fiber_separation
SKY_IFU_RADIUS_ARCSEC = np.sqrt(np.pi*(FIBER_RADIUS_ARCSEC/2)**2*60/np.pi) 
# only approximate radius, in arcsec, given 37" fibers and 60 in the bundle

# because I am suspicious that computing separations with SkyCoord is too slow
def deg2rad(degrees):
    return degrees*np.pi/180.

def rad2deg(radians):
    return radians*180./np.pi

def sphdist (ra1, dec1, ra2, dec2):
# measures the spherical distance in degrees
# The input has to be in degrees too
    dec1_r = deg2rad(dec1)
    dec2_r = deg2rad(dec2)
    return 2*rad2deg(np.arcsin(np.sqrt((np.sin((dec1_r - dec2_r) / 2))**2 + np.cos(dec1_r) * np.cos(dec2_r) * (np.sin((deg2rad(ra1 - ra2)) / 2))**2)))

In [ ]:
# running this search takes a minute or two
if True:
    search_string= "SELECT * FROM gaiadr3.gaia_source as gaia JOIN gaiadr3.astrophysical_parameters AS gaia_ap USING (source_id) WHERE gaia.phot_g_mean_mag <= 9 and gaia.phot_g_mean_mag > 5 AND gaia.parallax_error/parallax < 0.2 AND gaia_ap.teff_gspphot > 6000 and gaia_ap.teff_gspphot < 7500 AND gaia.phot_variable_flag != 'VARIABLE'"
    j = Gaia.launch_job_async(search_string)
    ftype = j.get_results()
    ftype.write('broad_search-result.xml',format='votable',overwrite=True)
    #ftype.write('broad_search-result.fits',format='fits',overwrite=True) # cannot get fits table to save...

    # establish new columns with initial value of 40
    ftype['BG_RP_SB']=ftype['phot_g_mean_mag']*0.+40.
    ftype['BG_G_SB']=ftype['phot_g_mean_mag']*0.+40.
    ftype['BG_BP_SB']=ftype['phot_g_mean_mag']*0.+40.
else:
    ftype = Table.read('broad_search-result.xml')



In [ ]:
ftype.colnames

In [ ]:
plt.scatter(ftype['teff_gspphot'],ftype['teff_gspspec'],marker='.',alpha=0.4)
plt.xlim(5800,7700)
plt.ylim(4000,8000)
plt.xlabel('teff_gspphot [K]')
plt.ylabel('teff_gspspec [K]')

In [ ]:
c = SkyCoord(frame="icrs", ra=ftype['ra'], dec=ftype['dec'],unit='deg')
c_g=c.transform_to('galactic')

#iis=(v3_cross['teff_gspspec'] > 0) 
#iis=(ftype['teff_gspspec'] > 6000) & (v3_cross['teff_gspspec'] < 7500)
#iis=(ftype['BG_SB'] == 40) # all of them
iis=(np.abs(c_g.b.deg) < 1) # all of them

fig = plt.figure(figsize=(14, 6))
#ax = plt.subplot(111, projection="aitoff")
#ugh_aitoff_plot(ax,ls,bs,intens,0,10,'Ha Intensity [R]')
#ax.scatter(c_g.l.deg,c_g.b.deg,s=3,c='r')
plt.scatter(c_g.l.deg,c_g.b.deg,s=1,c='b')
plt.scatter(c_g[iis].l.deg,c_g[iis].b.deg,s=3,c='r')
plt.xlabel('l [deg]')
plt.ylabel('b [deg]')

In [9]:
# double check the stellar background at sources 
def find_gaia_sb(rasdecs0):
    gaia_limiting_mag = 21
    
    ra = rasdecs0.ra.deg
    dec = rasdecs0.dec.deg
    
#    j = Gaia.launch_job_async("SELECT source_id, ra,dec,phot_g_mean_mag,phot_g_mean_mag FROM gaiadr2.gaia_source WHERE phot_g_mean_mag <= "+str(gaia_limiting_mag)+" AND 1=CONTAINS(POINT('ICRS',ra,dec), CIRCLE('ICRS',"+str(ra)+","+str(dec)+", 0.05))")
    j = Gaia.launch_job_async("SELECT source_id, ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag FROM gaiadr3.gaia_source WHERE phot_rp_mean_mag <= "+str(gaia_limiting_mag)+" AND 1=CONTAINS(POINT('ICRS',ra,dec), CIRCLE('ICRS',"+str(ra)+","+str(dec)+", 0.05))")
    cat0 = j.get_results()
    ddgaia=sphdist(ra,dec,cat0['ra'],cat0['dec'])    
#    criteria = (ddgaia < SKY_IFU_RADIUS_ARCSEC/3600.) 
    criteria = (ddgaia < FIBER_RADIUS_ARCSEC/3600.) & (ddgaia > 1/3600.)
    cat = cat0[criteria]

    if len(cat) > 0:
        mag_sum_g = -2.5*np.log10(np.sum(10**(-0.4*cat['phot_g_mean_mag'])))
        mag_sum_bp = -2.5*np.log10(np.sum(10**(-0.4*cat['phot_bp_mean_mag'])))
        mag_sum_rp = -2.5*np.log10(np.sum(10**(-0.4*cat['phot_rp_mean_mag'])))
        gaia_sb_g = mag_sum_g + 2.5*np.log10(np.pi*(FIBER_RADIUS_ARCSEC/2.)**2)
        gaia_sb_bp = mag_sum_bp + 2.5*np.log10(np.pi*(FIBER_RADIUS_ARCSEC/2.)**2)
        gaia_sb_rp = mag_sum_rp + 2.5*np.log10(np.pi*(FIBER_RADIUS_ARCSEC/2.)**2)
    else:
        gaia_sb_g = 40 # some arbitrary high number?
        gaia_sb_bp = 40 # some arbitrary high number?
        gaia_sb_rp = 40 # some arbitrary high number?
    
    return gaia_sb_g,gaia_sb_bp,gaia_sb_rp



In [ ]:
# compute the background surface brightness based on gaia stars within 37" fiber size

if True: # Re-run background flux calculations?
    # this takes ~30m per ~2000 objects
    # ~4h for the whole sample
    
    #iis_MW=(ftype['BG_SB']==40) & (np.abs(c_g.b.deg) < 1)
    #print(np.sum(iis_MW))
    #sub = ftype[iis_MW]

    sub = ftype

    program_starts = time.time()



    #for i,coord in enumerate(c[iis_MW]):
    for i,coord in enumerate(c):
    #    print(i, coord)
        gaia_sb_g,gaia_sb_bp,gaia_sb_rp = find_gaia_sb(coord)
        sub['BG_G_SB'][i] = gaia_sb_g
        sub['BG_RP_SB'][i] = gaia_sb_rp
        sub['BG_BP_SB'][i] = gaia_sb_bp
    #    print(gaia_sb_g,gaia_sb_bp,gaia_sb_rp)
        #print(sub[i], i)
        if (i % 10 == 0):
            now = time.time()
            elapsed = now - program_starts
            print(f"It has been {elapsed} seconds since the loop started, loop {i}")

    ftype = sub

    ftype.write('broad_search-result_BG.xml',format='votable',overwrite=True)
else:
    ftype = Table.read('broad_search-result_BG.xml')

In [ ]:
# conversion of magnitude to SB for the faintest stars
mag =  -2.5*np.log10((10**(-0.4*9.)))
sb =  mag + 2.5*np.log10(np.pi*(FIBER_RADIUS_ARCSEC/2.)**2)
print(sb, mag)


In [ ]:
measured = (ftype['BG_RP_SB'] < 40) & (ftype['BG_BP_SB'] < 40) & (ftype['BG_G_SB'] < 40)
#plt.hist(ftype['BG_RP_SB'],label='RP')
plt.hist(ftype['BG_G_SB'][measured],label='G',histtype='step')
plt.hist(ftype['BG_RP_SB'][measured],label='RP',alpha=1,histtype='step')
plt.hist(ftype['BG_BP_SB'][measured],label='BP',alpha=1,histtype='step')
plt.xlabel('Surface brightness [mag/arcsec^2]')
plt.ylabel('N')
plt.axvline(18.5,linestyle='dashed',color='k',label='full moon')
plt.legend()



In [12]:
#mag =  -2.5*np.log10((10**(-0.4*9.)))
#sb =  mag + 2.5*np.log10(np.pi*(FIBER_RADIUS_ARCSEC/2.)**2)

flux_star = 10**(ftype['phot_rp_mean_mag']/(-2.5))
flux_bg = 10**((ftype['BG_RP_SB'] - 2.5*np.log10(np.pi*(FIBER_RADIUS_ARCSEC/2.)**2))/(-2.5))

flux_star_g = 10**(ftype['phot_g_mean_mag']/(-2.5))
flux_bg_g = 10**((ftype['BG_G_SB'] - 2.5*np.log10(np.pi*(FIBER_RADIUS_ARCSEC/2.)**2))/(-2.5))

In [ ]:
ii = (ftype['BG_RP_SB'] < 40)

#plt.scatter(flux_star[ii],flux_bg[ii]/flux_star[ii],alpha=0.2,marker='.')
plt.scatter(flux_bg_g[ii]/flux_star_g[ii],flux_bg[ii]/flux_star[ii],alpha=0.2,marker='.')
#plt.plot([0,.2],[0,.2])

plt.axhline(.01,color='red')
plt.axhline(.05,color='red',linestyle='dashed')

plt.axvline(.01,color='red')
plt.axvline(.05,color='red',linestyle='dashed')

plt.xlabel('fraction of background flux, G band')
plt.ylabel('fraction of background flux, RP band')
plt.yscale('log')
plt.xscale('log')

fig = plt.figure(figsize=(8, 6))
bins=np.arange(0,.15,.005)
plt.hist(flux_bg_g[ii]/flux_star_g[ii],histtype='step',label='G',bins=bins)
plt.hist(flux_bg[ii]/flux_star[ii],histtype='step',label='RP',bins=bins)
plt.axvline(0.01,linestyle='dashed',color='k',label='<1%')
plt.yscale('log')
plt.legend()
plt.xlabel('fraction of background flux')
plt.ylabel('N')
plt.xlim(0,0.12)

In [ ]:
contrast_01 = (ftype['BG_RP_SB'] == 40) | (flux_bg/flux_star < 0.01)

l_wrap = c_g.l.deg
wrappy = l_wrap > 180
l_wrap[wrappy] = l_wrap[wrappy] - 360

fig = plt.figure(figsize=(14, 6))
plt.scatter(l_wrap,c_g.b.deg,s=1,c='b',label='all F type')
plt.scatter(l_wrap[contrast_01],c_g[contrast_01].b.deg,s=3,c='r', label='isolated F type')
plt.xlim(180,-180)
plt.legend()
plt.xlabel('l [deg]')
plt.ylabel('b [deg]')

print(np.sum(contrast_01))

In [ ]:
# pick out some examples to look at in decaps
# at the 1% level:
# https://decaps.legacysurvey.org/viewer?ra=173.80547327081052&dec=-60.89464656421644&zoom=13
# at the 5% level:
# https://decaps.legacysurvey.org/viewer?ra=206.64719495036567&dec=-62.06946758588094&zoom=13
samp = (flux_bg/flux_star < 0.0101) & (flux_bg/flux_star > 0.009) & (np.abs(ftype['b']) < 1) & (ftype['l'] < 360) & (ftype['l'] > 270)
print(np.sum(samp))
print(ftype['ra'][samp],ftype['dec'][samp])

samp = (flux_bg/flux_star < 0.055) & (flux_bg/flux_star > 0.045) & (np.abs(ftype['b']) < 1) & (ftype['l'] < 360) & (ftype['l'] > 270)
print(np.sum(samp))
print(ftype['ra'][samp],ftype['dec'][samp])

samp = (flux_bg/flux_star < 0.6) & (flux_bg/flux_star > 0.4) & (np.abs(ftype['b']) < 1) & (ftype['l'] < 360) & (ftype['l'] > 270)
print(np.sum(samp))
print(ftype['ra'][samp],ftype['dec'][samp])


In [ ]:
ftype_v4 = ftype[contrast_01]

ll_arr = []
max_dist = []
median_dist =[]

for ll in np.arange(0,-180,-1):
    for bb in np.arange(-5,5,1):
        dd = sphdist(ftype_v4['l'],ftype_v4['b'],ll,bb)
        ss=dd[np.argsort(dd)]
        ll_arr.append(ll)
        max_dist.append(ss[11])
        median_dist.append(np.median(ss[0:11]))
        


In [ ]:
plt.scatter(ll_arr,max_dist,label='max')
plt.scatter(ll_arr,median_dist,color='r',alpha=0.5,label='median')
plt.xlabel('l [deg]')
plt.ylabel('separation [deg]')
plt.legend()
plt.xlim(0,-180)
plt.title('<1% contrast with background stars')

In [19]:
# 5% contrast
contrast_05 = (ftype['BG_RP_SB'] == 40) | (flux_bg/flux_star < 0.05)

ftype_v4_05 = ftype[contrast_05]

ll_arr_05 = []
max_dist_05 = []
median_dist_05 =[]

for ll in np.arange(0,-180,-1):
    for bb in np.arange(-5,5,1):
        dd = sphdist(ftype_v4_05['l'],ftype_v4_05['b'],ll,bb)
        ss=dd[np.argsort(dd)]
        ll_arr_05.append(ll)
        max_dist_05.append(ss[11])
        median_dist_05.append(np.median(ss[0:11]))
        


In [ ]:
plt.scatter(ll_arr_05,max_dist_05,label='max')
plt.scatter(ll_arr_05,median_dist_05,color='r',alpha=0.5,label='median')
plt.xlabel('l [deg]')
plt.ylabel('separation [deg]')
plt.legend()
plt.xlim(0,-180)
plt.title('<5% contrast with background stars')

In [ ]:
ftype_v4.write('Gaia_stars_5-9_ftype_v4.xml',format='votable',overwrite=True)


In [ ]:
ftype_v4 = Table.read('Gaia_stars_5-9_ftype_v4.xml')

In [ ]:
# keep only a few of the columns for a 'minimal' table that can be put into the ops DB
colnames = ftype_v4.colnames
colnames.remove('ra')
colnames.remove('dec')
colnames.remove('source_id')
colnames.remove('phot_g_mean_mag')
colnames.remove('phot_bp_mean_mag')
colnames.remove('phot_rp_mean_mag')
colnames.remove('BG_RP_SB')
colnames.remove('BG_G_SB')
colnames.remove('BG_BP_SB')
#colnames.remove('')

ftype_v4_min = ftype_v4.copy()

for col in colnames:
    del ftype_v4_min[col]

print(ftype_v4_min.colnames)

In [17]:
ftype_v4_min.write('Gaia_stars_5-9_ftype_v4_min.fits',format='fits',overwrite=True)
ftype_v4_min.write('Gaia_stars_5-9_ftype_v4_min.csv',format='csv',overwrite=True)
